In [ ]:
'''

語音轉文字小工具
https://github.com/reic/colab_python/blob/main/%E9%8C%84%E9%9F%B3%E6%AA%94%E8%BD%89%E6%96%87%E5%AD%97.ipynb

'''

In [ ]:
#url = 'https://player.soundon.fm/p/e75f72d6-a458-4982-861e-6c5fbad85956' #財報狗
#url = 'https://player.soundon.fm/p/d2aab16c-3a70-4023-b52b-e50f07852ecd' #財經M平方
url = 'https://player.soundon.fm/p/954689a5-3096-43a4-a80b-7810b219cef3' #Gooaye 股癌
file_name = 'Gooaye 股癌'


In [ ]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By



!pip install fake_useragent
from fake_useragent import UserAgent
from lxml import etree

import pandas as pd
import random
import time

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Ign:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
browser = webdriver.Chrome('chromedriver',chrome_options=chrome_options)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: use options instead of chrome_options
  """


In [ ]:
#下載網頁
browser.get(url)
for i in range(200):

  browser.execute_script("window.scrollBy(0, window.innerHeight)")
  t = random.uniform(0.75, 1.5) #隨機數
  time.sleep(t) #暫停


In [ ]:
#擷取頻道目錄
channel_name = [] #裝頻道名稱
title = [] #裝標題
date = [] #裝日期
links = [] #裝連結
time_long = [] #裝時間長度

#擷取網頁元素
temp_channel = browser.find_elements(By.XPATH,'/html/body/div[1]/div/div[1]/div[2]/div/div[1]/div/div[2]/div[2]/h3')[0].text
URLs = browser.find_elements(By.XPATH,'/html/body/div[1]/div/div[1]/div[2]/div/div[2]/div/div/a')
temp_title = browser.find_elements(By.XPATH,'/html/body/div[1]/div/div[1]/div[2]/div/div[2]/div/div/a/div/div[2]/h4/div')
temp_date = browser.find_elements(By.XPATH,'/html/body/div[1]/div/div[1]/div[2]/div/div[2]/div/div/a/div/div[2]/div[2]/time[2]')
temp_time = browser.find_elements(By.XPATH,'/html/body/div[1]/div/div[1]/div[2]/div/div[2]/div/div/a/div/div[2]/div[2]/time[1]') 
#/html/body/div[1]/div/div[1]/div[2]/div/div[2]/div/div/a/div/div[2]/div[2]/time[1]/text()
#擷取連結
for node in URLs:
  links.append(node.get_attribute("href"))
links = pd.DataFrame(links)

#擷取標題、出刊日、長度
t = len(URLs)
for i in range(len(URLs)):
  channel_name.append(temp_channel)
  date.append(temp_date[i].text)
  title.append(temp_title[i].text)
  time_long.append(str(temp_time[i].text))



#整理為表格
channel_name = pd.DataFrame(channel_name)
date = pd.DataFrame(date)
title = pd.DataFrame(title)
time_long = pd.DataFrame(time_long)

data = pd.concat([channel_name, date, title, time_long, links], axis=1)
data.columns = ['channel', 'date', 'title', 'long', 'link']

In [ ]:
#擷取單集摘要、音源網址
audio_link = []
Summary = []

for i in range(len(data['link'])):

  url = data['link'][i]
  #下載網頁
  browser.get(url)
  t = random.uniform(0.75, 1.5) #隨機數
  time.sleep(t) #暫停

  #取得播放連結
  play = browser.find_element_by_xpath('/html/body/div[1]/div/div[1]/div[2]/div/div[1]/div/div[3]/div/button')
  play.click()
  browser.implicitly_wait(0.05) #等待載入
  element = browser.find_elements_by_xpath('/html/body/audio')

  temp_audio = []
  for node in element:
    temp_audio.append(node.get_attribute("src"))
  audio_link.append(temp_audio)


  #取得摘要
  temp_Summary = browser.find_elements(By.XPATH,'/html/body/div[1]/div/div[1]/div[2]/div/div[2]/div/div')[0].text
  temp_Summary = temp_Summary.replace('\n', '')
  Summary.append(temp_Summary)

audio_link = pd.DataFrame(audio_link)
Summary = pd.DataFrame(Summary)




/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead


In [ ]:
data


,channel,date,title,long,link,audio_link,Summary
0,Gooaye 股癌,2022-04-09,EP234 | 🕴️,52:38,https://player.soundon.fm/p/954689a5-3096-43a4...,https://chtbl.com/track/4B4E1D/rss.soundon.fm/...,Vito Corleone “Cause a man who doesn't spend t...
1,Gooaye 股癌,2022-04-06,EP233 | 🛵,47:52,https://player.soundon.fm/p/954689a5-3096-43a4...,https://chtbl.com/track/4B4E1D/rss.soundon.fm/...,好甩要暈車了==Welcome on board ✈️Chicco Seat3 寶寶的安全頭...
2,Gooaye 股癌,2022-04-02,EP232 | 🧙,50:31,https://player.soundon.fm/p/954689a5-3096-43a4...,https://chtbl.com/track/4B4E1D/rss.soundon.fm/...,盤點一下本集節目由 台灣情趣領導商城 Dr.情趣 贊助Dr. 情趣，台灣情趣第一領導商城秉持...
3,Gooaye 股癌,2022-03-30,EP231 | 👽,51:14,https://player.soundon.fm/p/954689a5-3096-43a4...,https://chtbl.com/track/4B4E1D/rss.soundon.fm/...,QA 有厲害的摸高大師決定要進場了各位小心史上最偉大的發明~寶寶安撫神器來了來自韓國第一品牌...
4,Gooaye 股癌,2022-03-26,EP230 | 🀅,52:53,https://player.soundon.fm/p/954689a5-3096-43a4...,https://chtbl.com/track/4B4E1D/rss.soundon.fm/...,這是很不真實的一個禮拜本集節目由 # Sony LinkBuds WF-L900 真無線藍牙...
...,...,...,...,...,...,...,...
229,Gooaye 股癌,2020-03-04,EP5 | ∑( ◦д⊙)‼ 🔫,27:24,https://player.soundon.fm/p/954689a5-3096-43a4...,None,聯準會緊急降息兩碼，卻沒有有效的激勵美股持續上漲，這和前天大漲 1300 點的創紀錄漲點可能...
230,Gooaye 股癌,2020-03-03,EP4 |( ͡° ͜ʖ ͡°),25:27,https://player.soundon.fm/p/954689a5-3096-43a4...,https://chtbl.com/track/4B4E1D/rss.soundon.fm/...,GE 傳奇 Jack Welch 逝世，分享他的一些軼事。史上最大的反彈現蹤，是否空頭有結束...
231,Gooaye 股癌,2020-03-02,EP3 |( *・ω・)✄╰ひ╯,26:54,https://player.soundon.fm/p/954689a5-3096-43a4...,https://chtbl.com/track/4B4E1D/rss.soundon.fm/...,若疫情持續擴散至美國發生預防性裁員，那會是下一波的危機；中國 PMI 開出新低，如但如同新春...
232,Gooaye 股癌,2020-02-29,EP2 |( *・ω・)╰✄-╯,30:31,https://player.soundon.fm/p/954689a5-3096-43a4...,https://chtbl.com/track/4B4E1D/rss.soundon.fm/...,股市躺完換黃金躺，所謂的「避險產品」很多只是媒體教給你的。迪士尼的 Disney + 將是 ...


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


data = pd.concat([channel_name, date, title, time_long, links, audio_link, Summary], axis=1)
data.columns = ['channel', 'date', 'title', 'long', 'link', 'audio_link', 'Summary']


#輸出檔案
path = '/content/drive/MyDrive/DATA set/'+file_name+'.csv'
data.to_csv(path, index=None)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import requests


     |████████████████████████████████| 23.4 MB 1.2 MB/s 


In [ ]:



url = "https://anchor.fm/s/209541b8/podcast/play/47944792/https%3A%2F%2Fd3ctxlq1ktw2nl.cloudfront.net%2Fstaging%2F2022-1-20%2F249492821-44100-2-664ddbe3944b5.m4a"    # 音频的URL
req = requests.get(url) 


ValueError: ignored